In [2]:
from keras.models import Sequential
from keras.layers import Input,Conv2D,Lambda,LeakyReLU,Activation,Concatenate,MaxPooling2D,BatchNormalization,Flatten,Dense,Dropout,GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from matplotlib import pyplot as plt
from keras.applications.mobilenet import MobileNet
from keras.models import Model
from tensorflow.keras import metrics


import numpy as np #矩阵库
import cv2         #图像处理库
import matplotlib #绘图库
import pandas as pd      #数据处理库
import os
import sys
import random
from datetime import datetime
from subprocess import check_output

#设置全局变量
ImageTrainData={}
ImageValData={}

def loadTrainData(trainDir): #读取训练图像数据及处理
    global ImageTrainData
    
    images=os.listdir(trainDir)
    print("训练集图片共有"+str((len(images)))+"张")
    
    for imageFileName in images:
        imageFullPath=os.path.join(trainDir,imageFileName)
        img=cv2.imread(imageFullPath)
        arr=img
        #print(arr.shape)
          #提取各颜色空间并生成LGI颜色空间
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4,4))
        Blue=arr[:,:,0]
        Green=arr[:,:,1]
        Red=arr[:,:,2]
        #print(Green.shape)
        Green=clahe.apply(Green)
        #print(Green.shape)
        Lab=cv2.cvtColor(arr,cv2.COLOR_BGR2Lab)
        Light=Lab[:,:,0]
        Light=clahe.apply(Light)
        #print(Light.shape)
        I1=cv2.cvtColor(arr,cv2.COLOR_BGR2GRAY)
        #print(I1.shape)
        I1=clahe.apply(I1)
        arr=cv2.merge([Green,Light,I1])
        dim0=arr.shape[0]#高度
        dim1=arr.shape[1]#宽度
        dim2=arr.shape[2]#深度
        #调整图片尺寸统一
        if (dim0<128 or dim1<128 or dim2<3):
            print("图片错误"+str(arr.shape))
        arr=cv2.resize(arr,(128,128))
        dim0=arr.shape[0]#新高度
        dim1=arr.shape[1]#新宽度
        dim2=arr.shape[2]#新深度
        #确定修改完成
        if (dim0!=128 or dim1!=128 or dim2!=3):
            print("图片错误"+str(arr.shape))
        #print(arr)
        arr = np.array(arr, dtype="float") / 255.0
        imageFileName = imageFileName.replace('.jpg','')
        ImageTrainData [str(imageFileName)] = np.array(arr)
    print('完成')
    return

def loadValData(trainDir): #读取验证图像数据及处理
    global ImageValData
    
    images=os.listdir(trainDir)
    print("训练集图片共有"+str((len(images)))+"张")
    
    for imageFileName in images:
        imageFullPath=os.path.join(trainDir,imageFileName)
        img=cv2.imread(imageFullPath)
        arr=img
        #print(arr.shape)
          #提取各颜色空间并生成LGI颜色空间
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4,4))
        #Blue=arr[:,:,0]
        Green=arr[:,:,1]
        #Red=arr[:,:,2]
        #print(Green.shape)
        Green=clahe.apply(Green)
        #print(Green.shape)
        #Lab=cv2.cvtColor(arr,cv2.COLOR_BGR2Lab)
        #Light=Lab[:,:,0]
        #Light=clahe.apply(Light)
        #print(Light.shape)
        #I1=cv2.cvtColor(arr,cv2.COLOR_BGR2GRAY)
        #print(I1.shape)
        #I1=clahe.apply(I1)
        #arr=cv2.merge([Green,Light,I1])
        dim0=arr.shape[0]#高度
        dim1=arr.shape[1]#宽度
        dim2=arr.shape[2]#深度
        #调整图片尺寸统一
        if (dim0<200 or dim1<200 or dim2<3):
            print("图片错误"+str(arr.shape))
        arr=cv2.resize(arr,(200,200))
        dim0=arr.shape[0]#新高度
        dim1=arr.shape[1]#新宽度
        dim2=arr.shape[2]#新深度
        #确定修改完成
        if (dim0!=200 or dim1!=200 or dim2!=3):
            print("图片错误"+str(arr.shape))
        #print(arr)
        arr = np.array(arr, dtype="float") / 255.0
        imageFileName = imageFileName.replace('.jpg','')
        ImageValData [str(imageFileName)] = np.array(arr)
    print('完成')
    return

print("读取数据"+str(datetime.now()))
sys.stdout.flush()
loadTrainData("a. Training Set")
#loadValData("B. Disease Grading/1. Original Images/b. Testing Set")

def loadTrainLabels():#读取标签
    raw_df=pd.read_csv('a. IDRiD_Disease Grading_Training Labels.csv',sep=',',usecols=[0,1])
    row_count=raw_df.shape[0]#数据总量
    col_count=raw_df.shape[1]#列数
    print("共有"+str(row_count)+"数据\n"+"拥有"+str(col_count)+"特征")
    header_list=list(raw_df.columns)#检测特征含义
    print("特征分为:"+str(header_list))
    return raw_df

def loadValLabels():#读取标签
    raw_df=pd.read_csv('valstep1.csv',sep=',',usecols=[0,1])
    row_count=raw_df.shape[0]#数据总量
    col_count=raw_df.shape[1]#列数
    print("共有"+str(row_count)+"数据\n"+"拥有"+str(col_count)+"特征")
    header_list=list(raw_df.columns)#检测特征含义
    print("特征分为:"+str(header_list))
    return raw_df
print('读取标签')
Train_label=loadTrainLabels()
#Val_label=loadValLabels()
Train_keepImages =list(ImageTrainData.keys())
#Val_keepImages =list(ImageValData.keys())
print("完成")

train_image_arr=[]
train_data_arr=[]
val_image_arr=[]
val_data_arr=[]
print('合并图片与标签')
for index, row in Train_label.iterrows():
    key = str(row[0])
    if key in ImageTrainData:
        train_image_arr.append(key)
        train_data_arr.append(np.array(ImageTrainData [key]))
Train_label2 = pd.DataFrame({'Image name': train_image_arr, 'data': train_data_arr})

#for index, row in Val_label.iterrows():
#    key = str(row[0])
#    if key in ImageTrainData:
#        val_image_arr.append(key)
#        val_data_arr.append(np.array(ImageValData [key]))
#Val_label2 = pd.DataFrame({'Image name': val_image_arr, 'data': val_data_arr})

Train_label = pd.merge(Train_label2, Train_label, left_on='Image name', right_on='Image name', how='outer')
label_header_list1 = list(Train_label.columns)
#Val_label = pd.merge(Val_label2, Val_label, left_on='Image name', right_on='Image name', how='outer')
#label_header_list2 = list(Val_label.columns)

print(label_header_list1)
#print(label_header_list2)
print('训练集有', str(len(Train_label))+"张图片")
#print('测试集有', str(len(Val_label))+"张图片")
print("完成")
print('训练集排序')
trainset = Train_label.reset_index(drop=True)
#valset = Val_label.reset_index(drop=True)

print(trainset.head())
#print(valset.head())
print("完成")

trainX = trainset['data']
trainY = trainset['Retinopathy grade']

#valX = valset['data']
#valY = valset['Retinopathy grade']

print('训练集数量为', trainX.shape[0])
print('进行热独编码')
trainY =  to_categorical(trainY, num_classes=5)
#valY =  to_categorical(valY, num_classes=2)

print('完成')

Xtrain = np.zeros([trainX.shape[0],128,128,3])
for i in range(trainX.shape[0]):
    Xtrain[i] = trainX[i]
print(Xtrain.shape)

print("数据加强模板")
sys.stdout.flush()
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1, \
    height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,\
    horizontal_flip=True, fill_mode="nearest")
print("完成")

读取数据2020-03-28 00:25:30.395225
训练集图片共有413张
完成
读取标签
共有413数据
拥有2特征
特征分为:['Image name', 'Retinopathy grade']
完成
合并图片与标签
['Image name', 'data', 'Retinopathy grade']
训练集有 413张图片
完成
训练集排序
  Image name                                               data  \
0  IDRiD_001  [[[0.011764705882352941, 0.011764705882352941,...   
1  IDRiD_002  [[[0.03137254901960784, 0.0196078431372549, 0....   
2  IDRiD_003  [[[0.011764705882352941, 0.011764705882352941,...   
3  IDRiD_004  [[[0.011764705882352941, 0.011764705882352941,...   
4  IDRiD_005  [[[0.011764705882352941, 0.011764705882352941,...   

   Retinopathy grade  
0                  3  
1                  3  
2                  2  
3                  3  
4                  4  
完成
训练集数量为 413
进行热独编码
完成
(413, 128, 128, 3)
数据加强模板
完成


In [39]:
def loadTrainLabels():#读取标签
    raw_df=pd.read_csv('trainstep3.csv',sep=',',usecols=[0,1])
    row_count=raw_df.shape[0]#数据总量
    col_count=raw_df.shape[1]#列数
    print("共有"+str(row_count)+"数据\n"+"拥有"+str(col_count)+"特征")
    header_list=list(raw_df.columns)#检测特征含义
    print("特征分为:"+str(header_list))
    return raw_df

def loadValLabels():#读取标签
    raw_df=pd.read_csv('valstep1.csv',sep=',',usecols=[0,1])
    row_count=raw_df.shape[0]#数据总量
    col_count=raw_df.shape[1]#列数
    print("共有"+str(row_count)+"数据\n"+"拥有"+str(col_count)+"特征")
    header_list=list(raw_df.columns)#检测特征含义
    print("特征分为:"+str(header_list))
    return raw_df

Train_label=loadTrainLabels()
#Val_label=loadValLabels()
Train_keepImages =list(ImageTrainData.keys())
#Val_keepImages =list(ImageValData.keys())
print("完成")


共有230数据
拥有2特征
特征分为:['Image name', 'Retinopathy grade']
完成


In [40]:
train_image_arr=[]
train_data_arr=[]
val_image_arr=[]
val_data_arr=[]

for index, row in Train_label.iterrows():
    key = str(row[0])
    if key in ImageTrainData:
        train_image_arr.append(key)
        train_data_arr.append(np.array(ImageTrainData [key]))
Train_label2 = pd.DataFrame({'Image name': train_image_arr, 'data': train_data_arr})

#for index, row in Val_label.iterrows():
#    key = str(row[0])
#    if key in ImageTrainData:
#        val_image_arr.append(key)
#        val_data_arr.append(np.array(ImageValData [key]))
#Val_label2 = pd.DataFrame({'Image name': val_image_arr, 'data': val_data_arr})

Train_label = pd.merge(Train_label2, Train_label, left_on='Image name', right_on='Image name', how='outer')
label_header_list1 = list(Train_label.columns)
#Val_label = pd.merge(Val_label2, Val_label, left_on='Image name', right_on='Image name', how='outer')
#label_header_list2 = list(Val_label.columns)

print(label_header_list1)
#print(label_header_list2)
print('训练集有', str(len(Train_label))+"张图片")
#print('测试集有', str(len(Val_label))+"张图片")
print("完成")


['Image name', 'data', 'Retinopathy grade']
训练集有 230张图片
完成


In [41]:
trainset = Train_label.reset_index(drop=True)
#valset = Val_label.reset_index(drop=True)

print(trainset.head())
#print(valset.head())
print("完成")


  Image name                                               data  \
0  IDRiD_001  [[[0.011764705882352941, 0.011764705882352941,...   
1  IDRiD_002  [[[0.03137254901960784, 0.0196078431372549, 0....   
2  IDRiD_003  [[[0.011764705882352941, 0.011764705882352941,...   
3  IDRiD_004  [[[0.011764705882352941, 0.011764705882352941,...   
4  IDRiD_009  [[[0.011764705882352941, 0.011764705882352941,...   

   Retinopathy grade  
0                  2  
1                  2  
2                  1  
3                  2  
4                  2  
完成


In [42]:
trainX = trainset['data']
trainY = trainset['Retinopathy grade']

#valX = valset['data']
#valY = valset['Retinopathy grade']

print('训练集为', trainX.shape[0])

训练集为 230


In [43]:
trainY =  to_categorical(trainY, num_classes=3)
#valY =  to_categorical(valY, num_classes=2)

print('完成')
print(trainY[0])

完成
[0. 0. 1.]


In [44]:
Xtrain = np.zeros([trainX.shape[0],128,128,3])
for i in range(trainX.shape[0]):
    Xtrain[i] = trainX[i]
print(Xtrain.shape)

#Xval = np.zeros([valX.shape[0],200,200,3])
#for i in range(valX.shape[0]):
#    Xval[i] = valX[i]
#print(Xval.shape)


(230, 128, 128, 3)


In [45]:
import tensorflow as tf

from keras.applications.inception_v3 import InceptionV3

model_den=MobileNet(include_top=False,weights="imagenet",input_shape=(128,128,3))
x = model_den.output
x = GlobalAveragePooling2D()(x)
x= Dense(512, activation='tanh')(x)
predictions = Dense(3, activation='softmax')(x)
model_final = Model(inputs=model_den.input, outputs=predictions)
model_final.summary()
model_final.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=[metrics.categorical_accuracy])
print("完成")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 129, 129, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 64, 64, 32)        864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 64, 64, 32)        128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 64, 64, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 64, 64, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 64, 64, 32)        128       
__________

In [46]:
print("生成数据")
sys.stdout.flush()
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1, \
    height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,\
    horizontal_flip=True, fill_mode="nearest")
print("完成")



生成数据
完成


In [47]:
print("训练模型.")
sys.stdout.flush()

H = model_final.fit_generator(aug.flow(Xtrain, trainY, batch_size=16), \
    #validation_data=aug.flow(Xval, valY,batch_size=16), \
    #validation_steps=len(Xval)//16, \
    steps_per_epoch=len(trainX)//16, \
    epochs=300, verbose=1)


print("存储模型")
sys.stdout.flush()
model_final.save("MobileNetstep3.h5")
print("训练完成")

训练模型.
Epoch 1/300
14/14 [==============================] - 24s 2s/step - loss: 2.5720 - categorical_accuracy: 0.4955
Epoch 2/300
14/14 [==============================] - 17s 1s/step - loss: 0.9105 - categorical_accuracy: 0.6343
Epoch 3/300
14/14 [==============================] - 17s 1s/step - loss: 0.8284 - categorical_accuracy: 0.6323
Epoch 4/300
14/14 [==============================] - 17s 1s/step - loss: 0.8506 - categorical_accuracy: 0.6099
Epoch 5/300
14/14 [==============================] - 16s 1s/step - loss: 0.6915 - categorical_accuracy: 0.6861
Epoch 6/300
14/14 [==============================] - 18s 1s/step - loss: 0.8049 - categorical_accuracy: 0.6458
Epoch 7/300
14/14 [==============================] - 19s 1s/step - loss: 0.6059 - categorical_accuracy: 0.7264
Epoch 8/300
14/14 [==============================] - 17s 1s/step - loss: 0.7220 - categorical_accuracy: 0.7105
Epoch 9/300
14/14 [==============================] - 16s 1s/step - loss: 0.7232 - categorical_accuracy: 0.

14/14 [==============================] - 18s 1s/step - loss: 0.1893 - categorical_accuracy: 0.9458
Epoch 147/300
14/14 [==============================] - 17s 1s/step - loss: 0.2190 - categorical_accuracy: 0.9483
Epoch 148/300
14/14 [==============================] - 17s 1s/step - loss: 0.2312 - categorical_accuracy: 0.9458
Epoch 149/300
14/14 [==============================] - 18s 1s/step - loss: 0.1770 - categorical_accuracy: 0.9572
Epoch 150/300
14/14 [==============================] - 16s 1s/step - loss: 0.0817 - categorical_accuracy: 0.9617
Epoch 151/300
14/14 [==============================] - 17s 1s/step - loss: 0.0631 - categorical_accuracy: 0.9732
Epoch 152/300
14/14 [==============================] - 16s 1s/step - loss: 0.1911 - categorical_accuracy: 0.9552
Epoch 153/300
14/14 [==============================] - 16s 1s/step - loss: 0.0663 - categorical_accuracy: 0.9776
Epoch 154/300
14/14 [==============================] - 16s 1s/step - loss: 0.0675 - categorical_accuracy: 0.99

14/14 [==============================] - 16s 1s/step - loss: 0.1184 - categorical_accuracy: 0.9731
Epoch 291/300
14/14 [==============================] - 17s 1s/step - loss: 0.0530 - categorical_accuracy: 0.9866
Epoch 292/300
14/14 [==============================] - 17s 1s/step - loss: 0.1478 - categorical_accuracy: 0.9552
Epoch 293/300
14/14 [==============================] - 18s 1s/step - loss: 0.0749 - categorical_accuracy: 0.9642
Epoch 294/300
14/14 [==============================] - 18s 1s/step - loss: 0.1969 - categorical_accuracy: 0.9552
Epoch 295/300
14/14 [==============================] - 18s 1s/step - loss: 0.1175 - categorical_accuracy: 0.9731
Epoch 296/300
14/14 [==============================] - 17s 1s/step - loss: 0.0859 - categorical_accuracy: 0.9662
Epoch 297/300
14/14 [==============================] - 18s 1s/step - loss: 0.0502 - categorical_accuracy: 0.9910
Epoch 298/300
14/14 [==============================] - 17s 1s/step - loss: 0.2249 - categorical_accuracy: 0.95